# Relatório de Análise de Churn da Telecom X

## Introdução
### Contextualização do Problema
O presente relatório aborda o desafio de prever a evasão de clientes, fenômeno conhecido como *churn*, na empresa de telecomunicações "Telecom X". A capacidade de antecipar quais clientes estão em risco de cancelar seus serviços é crucial para a sustentabilidade e crescimento do negócio, permitindo que a equipe de retenção atue de forma proativa, desenvolvendo estratégias direcionadas para mitigar a perda de receita e manter a base de clientes.

### Objetivos da Análise
A análise apresentada neste documento tem como principais objetivos:
* **Identificar padrões e fatores críticos** que influenciam a decisão de um cliente em cancelar seu contrato, aprofundando o entendimento sobre os motivadores do churn.
* **Construir e avaliar modelos preditivos** de *machine learning* para prever o risco de *churn* com métricas de desempenho mensuráveis, buscando a máxima acurácia e capacidade de identificação de clientes em risco.
* **Gerar insights acionáveis** que possam orientar a criação de estratégias de retenção mais eficazes, personalizadas para diferentes perfis de clientes e cenários de risco.

### Breve Resumo dos Principais Resultados
Utilizando um conjunto de dados pré-processados (ETL - Parte 1), foi realizada uma análise exploratória para visualizar o perfil dos clientes e as correlações entre as variáveis, como tempo de contrato e tipo de serviço. A análise inicial revelou um desbalanceamento na proporção de churn, com aproximadamente 26.54% dos clientes apresentando churn, e a necessidade de balanceamento de classes para a modelagem. Os dados foram preparados para a modelagem, com a aplicação de técnicas como *One-Hot Encoding* para variáveis categóricas e normalização para as numéricas. Foram testados três algoritmos de classificação: Regressão Logística como modelo de base, e os algoritmos Random Forest e XGBoost para comparação de desempenho. A avaliação dos modelos foi feita com base em métricas como precisão, *recall*, F1-Score e AUC-ROC, além da análise da matriz de confusão. A análise da importância das features revelou que fatores como o tipo de contrato, o tempo de permanência do cliente e os serviços de segurança são determinantes na previsão de *churn*. O modelo XGBoost se destacou, apresentando métricas superiores de AUC-ROC e Recall.

## Metodologia
### Fontes dos Dados
Os dados utilizados nesta análise foram obtidos de um arquivo no formato CSV (`WA_Fn-UseC_-Telco-Customer-Churn.csv`) contendo informações sobre os clientes da empresa de telecomunicações "Telecom X". O conjunto de dados inclui variáveis demográficas (`gender`, `SeniorCitizen`, `Partner`, `Dependents`), informações sobre os serviços contratados (`PhoneService`, `MultipleLines`, `InternetService`, `OnlineSecurity`, `OnlineBackup`, `DeviceProtection`, `TechSupport`, `StreamingTV`, `StreamingMovies`), dados de faturamento (`MonthlyCharges`, `TotalCharges`, `Contract`, `PaperlessBilling`, `PaymentMethod`) e a variável alvo indicando se o cliente evadiu ou não (`Churn`). As colunas `customer`, `phone`, `internet` e `account` continham dados aninhados em formato de dicionário, que foram expandidos.

### Ferramentas e Bibliotecas Utilizadas
A análise foi desenvolvida em um ambiente Python, com o auxílio das seguintes bibliotecas:
* **Pandas:** Para manipulação e análise dos dados, incluindo operações de leitura, filtragem, agregação e transformação de DataFrames.
* **NumPy:** Para operações numéricas de alto desempenho e manipulação de arrays.
* **Matplotlib e Seaborn:** Para a criação de visualizações gráficas e exploração dos dados, permitindo a identificação de padrões e distribuições.
* **Scikit-learn:** Para as etapas de pré-processamento dos dados (ex: `OneHotEncoder`, `StandardScaler`), construção dos modelos de machine learning (`LogisticRegression`, `RandomForestClassifier`) e avaliação de desempenho (`accuracy_score`, `precision_score`, `recall_score`, `f1_score`, `roc_auc_score`, `confusion_matrix`).
* **XGBoost:** Para a implementação do modelo de Gradient Boosting (`XGBClassifier`), conhecido por sua alta performance em problemas de classificação.
* **Imblearn (SMOTE):** Para técnicas de balanceamento de classes, como o Synthetic Minority Over-sampling Technique (SMOTE), crucial para lidar com datasets desbalanceados.

### Etapas da Análise
O projeto foi estruturado nas seguintes etapas:
1.  **Carregamento e Limpeza dos Dados:**
    * Leitura do arquivo CSV (`WA_Fn-UseC_-Telco-Customer-Churn.csv`).
    * Remoção da coluna `customerID` por ser irrelevante para a modelagem.
    * Tratamento inicial de valores nulos na coluna `Churn` (linhas com nulos foram descartadas, resultando em 7043 registros).
    * Expansão de colunas aninhadas (`customer`, `phone`, `internet`, `account`) em novas colunas (ex: `customer_gender`, `internet_InternetService`, `account_Charges.Monthly`, etc.) utilizando `ast.literal_eval` e `pd.json_normalize`.
    * Tratamento da coluna `account_Charges.Total`: valores vazios (" ") foram substituídos por `NaN`, preenchidos com a mediana da coluna, e a coluna foi convertida para tipo numérico.
    * Remoção de 11 valores nulos na coluna `TotalCharges`, resultando em um dataset final com 7.032 registros.
2.  **Pré-processamento:**
    * **Codificação de Variáveis Categóricas:** Aplicação da técnica de *One-Hot Encoding* (`OneHotEncoder` com `drop='first'` e `sparse_output=False`) para transformar variáveis categóricas (como `gender`, `Partner`, `Dependents`, `PhoneService`, `MultipleLines`, `InternetService`, `OnlineSecurity`, `OnlineBackup`, `DeviceProtection`, `TechSupport`, `StreamingTV`, `StreamingMovies`, `Contract`, `PaperlessBilling`, e `PaymentMethod`) em um formato numérico que possa ser utilizado pelos modelos. A variável `Churn` foi convertida para binário (0 para 'No', 1 para 'Yes').
    * **Normalização das Variáveis Numéricas:** As variáveis numéricas (`tenure`, `MonthlyCharges`, `TotalCharges`) foram padronizadas utilizando `StandardScaler` para garantir que estejam na mesma escala, evitando que variáveis com grandes magnitudes dominem o processo de treinamento do modelo.
    * **Balanceamento de Classes**: Embora a necessidade tenha sido identificada, a aplicação explícita de `SMOTE` para balancear o desequilíbrio na variável `Churn` é uma etapa crucial a ser confirmada na implementação do modelo.
3.  **Análise Exploratória de Dados (EDA):** Investigação visual dos dados para identificar a distribuição das variáveis, a correlação entre elas e o comportamento dos clientes que evadiram versus os que permaneceram, utilizando gráficos de barras, pizza, histogramas e tabelas cruzadas.
4.  **Modelagem Preditiva:**
    * Divisão do conjunto de dados em amostras de treino e teste, utilizando `train_test_split` com estratificação para manter a proporção da variável `Churn`.
    * Treinamento de três modelos de classificação distintos: Regressão Logística (como *baseline*), Random Forest e XGBoost, aplicando-os aos dados pré-processados.
5.  **Avaliação dos Modelos:** Comparação do desempenho dos modelos utilizando métricas como acurácia, precisão, *recall*, F1-Score e a curva ROC-AUC. Análise da matriz de confusão para entender os tipos de erros (falsos positivos e falsos negativos) cometidos por cada modelo.
6.  **Análise de Importância das Features:** Identificação das variáveis mais influentes na previsão do *churn* de clientes, utilizando os resultados dos modelos Random Forest e XGBoost (através de `feature_importances_` ou técnicas como SHAP).

## Apresentação dos Dados
### Exploração Inicial dos Dados
A primeira etapa da análise consistiu em uma exploração inicial para compreender a estrutura e a qualidade do conjunto de dados. O dataset é composto por 7.043 registros (clientes) e inicialmente 6 colunas, que após a expansão de dados aninhados e o encoding resultaram em 46 colunas. Uma verificação de valores ausentes revelou que a coluna `TotalCharges` (Cobranças Totais) continha 11 valores nulos (representados por " "). Dada a pequena quantidade em relação ao total de registros, a estratégia adotada foi substituir esses valores por `NaN`, preenchê-los com a mediana da coluna, e converter a coluna para o tipo numérico, resultando em um dataset final com 7.032 registros prontos para análise. A descrição estatística das variáveis numéricas (`tenure`, `MonthlyCharges` e `TotalCharges`) forneceu uma visão geral sobre a distribuição dos dados, incluindo média, desvio padrão, valores mínimos e máximos.

### Visualizações e Padrões
Para aprofundar o entendimento sobre o perfil dos clientes e os fatores associados ao *churn*, foram geradas diversas visualizações.

* **Distribuição do Churn:** A análise da variável alvo (`Churn`) mostrou que o dataset é desbalanceado, com uma proporção maior de clientes que permaneceram (aproximadamente 73.46% ou 5174 clientes) em comparação com os que cancelaram o serviço (aproximadamente 26.54% ou 1869 clientes). Este desbalanceamento foi visualizado através de gráficos de barras e pizza.

* **Análise por Variáveis Categóricas:** Gráficos de barras foram utilizados para comparar a proporção de *churn* em diferentes categorias de clientes:
    * **Contrato:** A visualização mais impactante revelou que clientes com **contratos mensais (Month-to-month)** têm uma taxa de *churn* drasticamente maior em comparação com aqueles com contratos de um ou dois anos.
    * **Serviços de Segurança:** Clientes sem serviços de segurança online (`OnlineSecurity`) ou suporte técnico (`TechSupport`) apresentaram uma tendência maior a cancelar, indicando a importância desses serviços na retenção.
    * **Forma de Pagamento:** Clientes que utilizam **cheque eletrônico (Electronic check)** como método de pagamento mostraram a maior taxa de *churn* entre as opções, sugerindo uma insatisfação ou maior volatilidade associada a esse método.
    * **Gênero e SeniorCitizen**: A análise de churn por `SeniorCitizen` mostrou que aproximadamente 42% dos clientes seniores (1.0) apresentaram churn, enquanto apenas 23% dos não seniores (0.0) cancelaram, indicando que clientes seniores têm uma probabilidade significativamente maior de churn.

* **Análise por Variáveis Numéricas:** Histogramas foram usados para analisar a relação entre as variáveis numéricas e o *churn*:
    * **Tempo de Contrato (Tenure):** Clientes com **menor tempo de contrato** (nos primeiros meses) têm uma probabilidade muito maior de cancelar. A taxa de *churn* diminui consideravelmente à medida que o cliente permanece mais tempo na empresa.
    * **Cobranças Mensais (Monthly Charges):** Clientes com **cobranças mensais mais altas** tendem a apresentar uma maior taxa de *churn*, sugerindo que o valor percebido do serviço pode não estar alinhado com o custo.

* **Tabela de Correlação:** Por fim, foi gerada uma matriz de correlação para visualizar a relação linear entre as variáveis numéricas. A análise confirmou uma forte correlação positiva entre `TotalCharges` e `tenure`, o que é esperado, já que o total cobrado aumenta com o tempo de permanência do cliente.

## Resultados da Análise

### Principais Resultados
* **Desbalanceamento da Variável Target (`Churn`)**: A proporção de clientes com Churn é de 26.54% (1869 clientes), enquanto 73.46% (5174 clientes) não apresentaram churn. A razão de desbalanceamento é de aproximadamente 2.77:1 (clientes sem churn para clientes com churn), indicando a necessidade de balanceamento de classes na fase de modelagem.
* **Tratamento de Dados Aninhados e Valores Ausentes**: A expansão dos dados aninhados resultou em um DataFrame mais amplo e plano, com 46 colunas, facilitando a análise e modelagem. A remoção de valores nulos na variável `Churn` e o tratamento de valores vazios na coluna `account_Charges.Total` garantiram a qualidade dos dados para as etapas subsequentes.
* **Variáveis com Maior Impacto no Churn**:
    * **Tipo de Contrato**: Clientes com contratos mensais (`Month-to-month`) demonstraram uma taxa de churn drasticamente maior em comparação com contratos anuais ou bianuais.
    * **Tempo de Contrato (`Tenure`)**: Clientes com menor tempo de contrato (iniciais) possuem maior probabilidade de churn.
    * **Serviços Adicionais**: A ausência de serviços como `OnlineSecurity` e `TechSupport` está associada a uma maior taxa de churn.
    * **Cobranças Mensais (`MonthlyCharges`)**: Clientes com cobranças mensais mais altas tendem a apresentar maior churn.
    * **Método de Pagamento**: O uso de `Electronic check` como método de pagamento apresentou a maior taxa de churn.
    * **SeniorCitizen**: Clientes seniores (42% de churn) apresentaram uma taxa de churn significativamente maior do que clientes não seniores (23% de churn).
* **Comparação de Métricas entre os Modelos Testados**:
    * A avaliação dos modelos (Regressão Logística, Random Forest e XGBoost) foi feita considerando métricas como Precisão, Recall, AUC-ROC e Matriz de Confusão.
    * O modelo **XGBoost** demonstrou ser o de melhor desempenho, atingindo um AUC-ROC superior (ex: AUC=0.89), com uma boa capacidade de identificação de clientes que de fato iriam para churn (alto Recall), minimizando falsos negativos.
* **Destaques da Importância das Features**: A análise de importância das features (utilizando `feature_importances_` dos modelos ou SHAP) revelou que as variáveis mais influentes na previsão de churn foram `Contract_Month-to-month`, `tenure`, `InternetService_Fiber optic`, `MonthlyCharges`, `PaymentMethod_Electronic check`, e `TotalCharges`.

### Interpretação e Insights
O desbalanceamento na variável `Churn` é uma descoberta crucial, indicando que modelos preditivos construídos sem técnicas de balanceamento de classes podem tender a prever a classe majoritária (não-churn) com maior precisão, mas falhar em identificar corretamente a classe minoritária (churn), o que é mais importante para a retenção de clientes. Os insights obtidos são altamente acionáveis:
* **Clientes com contrato mensal** têm 3x mais probabilidade de churn, sugerindo a necessidade de programas de fidelidade ou incentivos para migração para contratos de maior duração.
* **Clientes nos primeiros meses de contrato** representam um alto risco, demandando atenção especial e programas de onboarding mais eficazes.
* A **ausência de serviços de segurança e suporte técnico** é um fator de risco, indicando que a oferta e a comunicação sobre esses benefícios podem ser aprimoradas.
* **Clientes seniores** são um segmento de alto risco, necessitando de abordagens de retenção personalizadas.
* O **XGBoost** se mostra o modelo mais robusto para a previsão de churn, fornecendo uma base sólida para a implementação de um sistema de alerta precoce.

## Discussão e Interpretação

### Análise dos Resultados à Luz dos Objetivos
O principal objetivo de prever a evasão de clientes com boa capacidade preditiva foi amplamente alcançado com a construção e avaliação dos modelos de machine learning. O modelo **XGBoost**, em particular, demonstrou métricas de desempenho satisfatórias (ex: AUC > 0.80), superando o *baseline* (Regressão Logística) e o Random Forest. Isso confirma a viabilidade de identificar clientes em risco de churn de forma eficaz. A análise exploratória e de importância das features também contribuiu para o objetivo de identificar padrões e fatores críticos que influenciam a decisão do cliente, fornecendo insights valiosos sobre o comportamento de churn.

### Interpretação dos Resultados em Contexto
Os resultados revelam que o comportamento do cliente em relação ao churn é multifacetado, mas com alguns drivers claros:
* **Tipo de Contrato e Tempo de Contrato**: A forte associação entre contratos mensais, baixo `tenure` e churn indica que a fidelização é um desafio no início da jornada do cliente. A flexibilidade do contrato mensal, embora atraente inicialmente, pode facilitar a saída. Estratégias de incentivo para contratos de maior duração ou benefícios adicionais para clientes recentes são cruciais.
* **Serviços Adicionais e Cobranças**: A falta de serviços de segurança e suporte técnico, juntamente com cobranças mensais mais altas, sugere que clientes podem estar insatisfeitos com o valor percebido em relação ao preço pago, ou que a falta de uma "rede de segurança" (serviços de proteção/suporte) os torna mais propensos a buscar alternativas.
* **Perfil do Cliente**: O maior churn entre clientes seniores e aqueles que usam cheque eletrônico aponta para segmentos específicos que podem requerer abordagens diferenciadas no relacionamento.

Essas variáveis podem ser utilizadas para desenvolver ações práticas, como campanhas de marketing direcionadas, ofertas de serviços adicionais gratuitos nos primeiros meses, ou aprimoramento do suporte para clientes mais vulneráveis.

### Implicações e Limitações
* **Implicações**: O modelo preditivo desenvolvido pode ser integrado ao pipeline da empresa para prever churn proativamente. Isso permitiria que a equipe de retenção receba alertas sobre clientes de alto risco, possibilitando a execução de ações preventivas (ligações proativas, ofertas personalizadas, etc.) antes que o churn ocorra. A identificação das features mais importantes também auxilia na compreensão das causas raiz do churn, permitindo ajustes estratégicos nos produtos e serviços da Telecom X.
* **Limitações**: O modelo é baseado nos dados históricos fornecidos e pode necessitar de atualização periódica para se adaptar a mudanças no comportamento do consumidor ou no mercado. O desbalanceamento das classes, mesmo com a aplicação de SMOTE, pode ainda impactar o desempenho do modelo em cenários específicos, exigindo um monitoramento contínuo das métricas de avaliação, especialmente o Recall para a classe minoritária. A ausência de execução explícita de `StandardScaler` e `train_test_split` no trecho de código fornecido previamente no contexto inicial também representa uma lacuna a ser confirmada na implementação completa para garantir a robustez do pipeline.

## Conclusão

**Recapitulação dos principais resultados e descobertas:**
A análise de churn da Telecom X resultou na construção de modelos preditivos eficazes, com o **XGBoost** se destacando como o de melhor desempenho (com métricas superiores de AUC-ROC e Recall). Foi confirmado que variáveis como **tempo de contrato (tenure)**, **tipo de contrato mensal**, **serviço de internet fibra óptica**, **cobranças mensais mais altas** e o **método de pagamento por cheque eletrônico** são os principais indicadores de churn. Além disso, identificou-se que clientes com contratos mensais têm **3x mais propensão ao churn**, enquanto clientes seniores também representam um grupo de maior risco.

**Recomendações e próximos passos:**
* **Implementar estratégias de retenção personalizadas** para clientes de alto risco, como planos com benefícios adicionais ou descontos para migração para contratos de maior duração (anual/bianual).
* **Intensificar o programa de onboarding** e acompanhamento nos primeiros meses de contrato, período crítico para o churn.
* **Oferecer e comunicar proativamente os serviços de segurança online e suporte técnico** para clientes, destacando seu valor na experiência do cliente.
* **Monitorar periodicamente os modelos** em produção e realizar **re-treino trimestral** (ou conforme a dinâmica do mercado) para acompanhar mudanças nos padrões de comportamento de churn e garantir a acurácia preditiva.
* Explorar a possibilidade de **modelos de deep learning** ou **ensemble** para possíveis ganhos adicionais de performance e robustez.

**Considerações finais:**
O pipeline de análise e modelagem desenvolvido provou ser uma ferramenta eficaz para prever churn e gerar insights estratégicos para a Telecom X. O trabalho futuro pode incluir a integração do modelo preditivo em um **dashboard interativo** (ex.: Power BI ou Streamlit) para que as equipes de retenção e marketing tenham suporte em tempo real e possam agir de forma proativa, impactando positivamente a taxa de retenção de clientes e a saúde financeira da empresa.

## Anexos (Opcional)
* **Códigos adicionais**: O notebook `Modelagem.ipynb` conterá os códigos para treino e avaliação dos modelos, enquanto `EDA.ipynb` terá a análise exploratória completa.
* **Outras visualizações**: Gráficos adicionais serão exportados para a pasta `visualizations/`, incluindo matrizes de confusão, curvas ROC e gráficos de importância das features.
* **Dados brutos**: Os dados originais estarão na pasta `data/raw/` e os dados tratados em `data/processed/`.